In [31]:
%matplotlib tk
import numpy as np
import sys
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random as rd
import copy as cp
from datetime import datetime as dt
#
sys.path.append('..')
from PyAstroPol import *
roundOffDisplay(4)

In [46]:
class KTTCoelostat(System):
    def __init__(self, Sun, Sun_disp):
        #
        self.Sun = Sun
        Dec = np.radians(self.Sun.Declination)
        HA = np.radians(self.Sun.HourAngle)
        Lat = np.radians(self.Sun.Latitude)
        PolarAxis = np.array([0, np.sin(Lat), np.cos(Lat)])
        self.PolarAxis = PolarAxis
        #
        self.Dist_EW = 830.0  # East-West distance
        self.Dist_ZN = 740.0  # North-South distance
        #
        A = -np.nan_to_num(np.abs(HA)/HA, nan=-1.0)*self.Dist_EW
        B = self.Dist_ZN
        SunPos = np.array([np.sin(HA)*np.cos(Dec), 
                           np.cos(HA)*np.cos(Dec)*np.cos(Lat) + np.sin(Dec)*np.sin(Lat), 
                           -np.cos(HA)*np.cos(Dec)*np.sin(Lat) + np.sin(Dec)*np.cos(Lat)])
        #
        TEMP = np.dot(SunPos, PolarAxis)
        a = TEMP**2-PolarAxis[2]**2
        b = -2*B*PolarAxis[1]*PolarAxis[2]
        c = (A**2+B**2)*TEMP**2 - B**2*PolarAxis[1]**2
#         self.a, self.b, self.c = a, b, c
        if (a == 0):
            C = -c/b
        else:
            C = (-b + np.nan_to_num(np.abs(Dec)/Dec, nan=1.0)*np.nan_to_num(np.sqrt(b**2-4*a*c)))/2.0/a
        self.Dist_NS = np.abs(C)
        #
        self.Center1 = np.array([-A, 0.0, -C])
        self.Center2 = np.array([0.0, B, 0.0])
        #
        self.Incidence1 = -SunPos
        self.Reflection1 = np.array([A, B, C])/np.linalg.norm(np.array([A, B, C]))
        self.Normal1 = (self.Reflection1-self.Incidence1)/np.linalg.norm(self.Reflection1-self.Incidence1)
        self.Incidence2 = self.Reflection1
        self.Reflection2 = np.array([0.0, -1.0, 0.0])
        self.Normal2 = (self.Reflection2-self.Incidence2)/np.linalg.norm(self.Reflection2-self.Incidence2)
        #
        self.Sun.makeOrigin(self.Center1-self.Sun.Distance*self.Incidence1)
        Sun_disp.makeOrigin(self.Center1-self.Sun.Distance*self.Incidence1)
        #
        self.M1 = Surface(600, Mirror=True, n2=2.16-7.18j)
        self.M1.pointToDirection(self.Normal1)
        self.M1.makeOrigin(self.Center1)
        #
        self.M2 = Surface(600, Mirror=True, n2=2.16-7.18j)
        self.M2.pointToDirection(self.Normal2)
        self.M2.makeOrigin(self.Center2)
        #
        self.Window = Detector(500.0)
        self.Window.rotateAboutX(90.0)
        System.__init__(self, self.Sun, [self.M1, self.M2], self.Window, dRays=Sun_disp)
        return

In [47]:
Sun = AstroSource(10000, Clear=500, Dec=10.0, HA=-50.0, Lat=10.23, Dist=5000.0)
Sun_disp = AstroSource(20, Clear=500, Dec=10.0, HA=-50.0, Lat=10.23, Dist=5000.0)
Coel = KTTCoelostat(Sun, Sun_disp)
Coel.propagateRays()

..\PyAstroPol\Surface.py:225: RuntimeWarning: invalid value encountered in true_divide
  dplus = (-b+np.sqrt(b**2-4.0*a*c))/2.0/a
..\PyAstroPol\Surface.py:226: RuntimeWarning: divide by zero encountered in true_divide
  dminus = (-b-np.sqrt(b**2-4.0*a*c))/2.0/a
..\PyAstroPol\Surface.py:225: RuntimeWarning: divide by zero encountered in true_divide
  dplus = (-b+np.sqrt(b**2-4.0*a*c))/2.0/a
..\PyAstroPol\Surface.py:226: RuntimeWarning: invalid value encountered in true_divide
  dminus = (-b-np.sqrt(b**2-4.0*a*c))/2.0/a
..\PyAstroPol\Surface.py:290: RuntimeWarning: divide by zero encountered in true_divide
  tCosines = (self.iRays.oCosines + self.nCosines*np.cos(self.iTheta))/np.sin(self.iTheta)
..\PyAstroPol\Surface.py:290: RuntimeWarning: invalid value encountered in true_divide
  tCosines = (self.iRays.oCosines + self.nCosines*np.cos(self.iTheta))/np.sin(self.iTheta)


In [37]:
Fig = plt.figure()                                             
Ax = Fig.add_subplot(111, projection='3d')
Coel.draw(Ax)
adjustAspect(Ax, 5000.0)

..\PyAstroPol\Surface.py:225: RuntimeWarning: invalid value encountered in true_divide
  dplus = (-b+np.sqrt(b**2-4.0*a*c))/2.0/a
..\PyAstroPol\Surface.py:226: RuntimeWarning: divide by zero encountered in true_divide
  dminus = (-b-np.sqrt(b**2-4.0*a*c))/2.0/a
..\PyAstroPol\Surface.py:225: RuntimeWarning: divide by zero encountered in true_divide
  dplus = (-b+np.sqrt(b**2-4.0*a*c))/2.0/a
..\PyAstroPol\Surface.py:226: RuntimeWarning: invalid value encountered in true_divide
  dminus = (-b-np.sqrt(b**2-4.0*a*c))/2.0/a
..\PyAstroPol\Surface.py:290: RuntimeWarning: divide by zero encountered in true_divide
  tCosines = (self.iRays.oCosines + self.nCosines*np.cos(self.iTheta))/np.sin(self.iTheta)
..\PyAstroPol\Surface.py:290: RuntimeWarning: invalid value encountered in true_divide
  tCosines = (self.iRays.oCosines + self.nCosines*np.cos(self.iTheta))/np.sin(self.iTheta)


In [48]:
MM, Tra = Coel.getSystemMuellerMatrix()
print('Mueller matrix is : \n', MM)

Mueller matrix is : 
 [[ 1.0000 -0.0001  0.0761 -0.0014]
 [-0.0614  0.5648 -0.8096 -0.1531]
 [ 0.0448  0.7793  0.5869 -0.2109]
 [-0.0014  0.2606 -0.0002  0.9624]]


..\PyAstroPol\Surface.py:225: RuntimeWarning: invalid value encountered in true_divide
  dplus = (-b+np.sqrt(b**2-4.0*a*c))/2.0/a
..\PyAstroPol\Surface.py:226: RuntimeWarning: divide by zero encountered in true_divide
  dminus = (-b-np.sqrt(b**2-4.0*a*c))/2.0/a


In [62]:
getMuellerRotationMatrix(MM, -50)

matrix([[ 1.0000, -0.0001,  0.0761, -0.0014],
        [-0.0335, -0.8655, -0.4374,  0.2343],
        [-0.0683,  0.4209, -0.8993, -0.1141],
        [-0.0014,  0.2606, -0.0002,  0.9624]])

In [63]:
np.sqrt(2160**2+2560**2)*.0065

21.771798272076655